In [2]:
using Logging, LinearAlgebra, Revise

In [3]:
using PorousMaterials, MOFun
global_logger(ConsoleLogger(stdout, Logging.Info));

# Example case: make IRMOF-1_one_ring with 2-acetylamido-terephthalate linker

## parent structure

In [4]:
parent_structure = Crystal("IRMOF-1_one_ring.cif")
strip_numbers_from_atom_labels!(parent_structure)
infer_bonds!(parent_structure, true)

┌ Info: Crystal IRMOF-1_one_ring.cif has  space group. I am converting it to P1 symmetry.
└         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.


## search moiety

In [5]:
s_moty = moiety("find-replace/2-!-p-phenylene");

#### nodes are sorted by order, except that R-group nodes are sent to the end of the list

Sorting the tagged nodes to the end may degrade performance for multi-nuclear replacement motifs.  This is a compromise made for the simplicity of aligning the moiety; it forces the isomorphism between the search moiety alignment mask and the replace moiety to keep a consistent node ordering with the isomorphism between the search moiety and the parent structure.

#### should write a test for tagged atoms being at the end

## replace moiety

In [18]:
r_moty = moiety("2-acetylamido-p-phenylene");

## substructure search results

In [7]:
search = s_moty ∈ parent_structure;

#### all 4 are in 1 location

## choose 1 at random for a replacement

In [8]:
s2p_isomorphism = search.results[rand(1:4)].isomorphism;

## subtract R group to get mask

In [9]:
"""
Returns a copy of moiety w/o R-group tagged atoms
"""
function remove_R_group(moiety::Crystal)::Crystal
    moty = deepcopy(moiety)
    r_indices = MOFun.filter_R_group(moty)
    not_r = [i for i in 1:length(moty.atoms.species) if ! (i in r_indices)]
    atoms = Atoms(moty.atoms.species[not_r], moty.atoms.coords[not_r])
    moty = Crystal(moiety.name, moiety.box, atoms, moiety.charges)
    infer_bonds!(moty, false)
    return moty
end
;

In [10]:
mask = remove_R_group(s_moty);

## align mask to replace moiety

In [20]:
m2r_isomorphism = (mask ∈ r_moty).results[1].isomorphism;

#### only 1 configuration, but will not always be so (heteroleptic multi-functionalization)

## now we have the correspondence between the search moiety and our target location, and between the search moiety mask and the replace moiety.  we can perform orthogonal Procrustes

In [12]:
function adjust_coordinates!(xtal::Crystal)
   for (i, node) in enumerate(1:length(xtal.atoms.species))
        dxf = xtal.atoms.coords.xf[:, node] - xtal.atoms.coords.xf[:, 1] # rebuild the image (fixing first node @ origin)
        nearest_image!(dxf)
        xtal.atoms.coords.xf[:, node] = dxf
    end
end
;

In [13]:
geometric_center(coords) = sum(coords, dims=2)[:] / 3;

In [14]:
function remove_path_prefix(name::String)::String
    s = split(name, '/')
    @debug "split" s[2]
    n = length(s)
    @debug n
    if n > 1
        return s[2]
    else
        return name
    end
end
;

In [15]:
function effect_replacement(xtal::Crystal, s_moty::Crystal, r_moty::Crystal, s2p_isomorphism::Array{Int}, m2r_isomorphism)::Crystal
    # adjust atomic coordinates to account for periodic boundaries
    xtal_subset = xtal.atoms[s2p_isomorphism] # adjust coords on this, to preserve rest of original crystal in unit cell
    adjust_coordinates!(xtal)
    adjust_coordinates!(s_moty)
    adjust_coordinates!(r_moty)
    
    # get frac coords of subsets of atoms in xtal, r_moty, s_mask
    xtal_subset_coords = xtal.atoms.coords.xf[:,s2p_isomorphism]
    r_moty_subset_coords = r_moty.atoms.coords.xf[:,m2r_isomorphism]
    r_indices = MOFun.filter_R_group(s_moty) # which atoms from s_moty are NOT in r_moty?
    s_mask_indices = [index for index in 1:length(s_moty.atoms.species) if !(index ∈ r_indices)]
    s_mask_coords = s_moty.atoms.coords.xf[:,s_mask_indices]
    
    # get frac coords of geometric center of xtal subset, moieties, mask
    xtal_subset_center = geometric_center(xtal_subset_coords)
    s_moty_center = geometric_center(s_moty.atoms.coords.xf)
    @debug "r_moty_subset" r_moty_subset_coords
    r_moty_subset_center = geometric_center(r_moty_subset_coords)
    s_mask_center = geometric_center(s_mask_coords)
    
    # translate centers to origin
    @debug "s_moty" s_moty.atoms.coords.xf s_moty_center
    s_moty.atoms.coords.xf .-= s_moty_center
    @debug "r_moty" r_moty.atoms.coords.xf r_moty_subset_center
    r_moty.atoms.coords.xf .-= r_moty_subset_center
    xtal.atoms.coords.xf .-= xtal_subset_center
    s_mask_coords .-= s_mask_center
    
    # get Cartesian clouds to align s_moty to xtal_subset
    @debug "Find A" s_moty.box.f_to_c s_moty.atoms.coords.xf
    A = s_moty.box.f_to_c * s_moty.atoms.coords.xf
    B = xtal.box.f_to_c * xtal.atoms.coords.xf[:,s2p_isomorphism]
    
    # find rotation matrix via orthogonal Procrustes in Cartesian space
    F = svd(A * B')
    rot_s2p = F.V * F.U' # rotation matrix that transforms s_moty onto xtal_subset
    
    # repeat to align r_moty_subset to s_moty mask
    # get Cartesian clouds
    A = r_moty.box.f_to_c * r_moty_subset_coords
    B = s_moty.box.f_to_c * s_mask_coords
    
    # find rotation matrix
    F = svd(A * B')
    rot_r2m = F.V * F.U' # rotation matrix that transforms r_moty_subset onto s_mask
    
    # transform r_moty according to rot_r2m
    xformd_r_moty_coords = rot_r2m * r_moty.box.f_to_c * r_moty.atoms.coords.xf
    
    # transform again by rot_s2p and translate to crystal location
    xformd_r_moty_atoms = Atoms{Cart}(length(r_moty.atoms.species), r_moty.atoms.species, Cart(rot_s2p * xformd_r_moty_coords .+ xtal.box.f_to_c * xtal_subset_center))
    
    # convert aligned positions to frac coords w.r.t crystal box
    r_coords = Frac(xformd_r_moty_atoms, xtal.box)
    
    # subtract s_moty isomorphic subset from xtal
    keep = [i for i in 1:length(xtal.atoms.species) if !(i ∈ s2p_isomorphism)]
    crystal_species = xtal.atoms.species[keep]
    crystal_coords = xtal.atoms.coords[keep]
    
    # concatenate transformed r_moty's coords/species w/ xtal's
    species = vcat(crystal_species, r_moty.atoms.species)
    @debug "concatenating coordinates" crystal_coords.xf r_coords.coords.xf
    coords = hcat(crystal_coords.xf, r_coords.coords.xf)
    @debug "atom data" species coords

    # return new crystal
    name = remove_extension(xtal) * "_find_" * remove_path_prefix(s_moty.name) * "_replace_" * r_moty.name
    atoms = Atoms(species, Frac(coords))
    crystal = Crystal(name, xtal.box, atoms, Charges{Frac}(0))
    wrap!(crystal)
    infer_bonds!(crystal, true)
    return crystal
end
;

In [21]:
altered_crystal = effect_replacement(parent_structure, s_moty, r_moty, s2p_isomorphism, m2r_isomorphism)

Name: IRMOF-1_one_ring_find_2-!-p-phenylene_replace_2-acetylamido-p-phenylene
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 25.832000 Å. b = 25.832000 Å, c = 25.832000 Å
	Volume of unit cell: 17237.492730 Å³

	# atoms = 58
	# charges = 0
	chemical formula: Dict(:N => 1,:Zn => 4,:H => 15,:O => 13,:C => 25)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [22]:
write_cif(altered_crystal, "2-acetamido-IRMOF-1-one-ring.cif")